Hi. I'm new at machine learning and data science and this is my first public notebook, so sorry if it's messy.

My notebook scales my data on both axis (0 & 1) and then "extends" the data on a 3 dimensional array, expanding every feature to the actual number of gene you have in the snippet by creating the list of all the possible combinations (A x M ; T x C ; G x D ; C x 1) 

This will allow my model, which is a 2D convnet to convolve the input on 2 dimensions. 
At first I tried to build a great model, but I ended having 0.90 accuracy on the test submission, will being able to have 0.98 on both training and validation with like 50 epochs. So I decided to train 30 "quicks" models on only 7 epochs (having around 0.95 accuracy), each of them having their weights initialized with a different seed to increase the "diversity", and then blend their results. 

I also supposed that any model which could give me a prediction where a certain category of bacteria gets a total number > 11000 or < 9000 is a bad model, so I droped it out (assuming that the repartition in the test set is similar to the train set, meaning each category of bacteria represents 1/10 of the total population, so 10000 elements)

I didn't test this notebook I'm publishing now (cause I've run out of daily subscriptions) but you can test it if you want (and please do change the params, I'm pretty sure this can do better). I obtained 0.93 with a similare approach (so it should be around this number, or maybe not?) but with "better" models (around 20 epochs each) but it seemed to be overfitting every time so... 

If you have any remark, please punish me in the comments. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
dataraw = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv",index_col=0)
datarawtest = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv",index_col=0)
#dataraw.drop_duplicates(inplace=True)
print(dataraw.shape)
dataraw = dataraw.sample(frac = 1)


In [ ]:
from sklearn import preprocessing as skp 

datanorm = pd.concat([dataraw.iloc[:,:-1],datarawtest])
datanorm = skp.scale(datanorm, axis = 1)
scalerraw = skp.StandardScaler()

scalerraw.fit(datanorm) 
datanorm = scalerraw.transform(datanorm) 
print(datanorm)

In [ ]:
namelist = dataraw["target"].unique()
j = 0
for i in namelist:
    print(i)
    dataraw['target'].replace(i, j, inplace = True)
    j+=1

In [ ]:
L = np.zeros((286, 4))
L[0,:] = [0, 0 ,0 ,10]
ind = 1
for k in range(10000):
    r = k
    M = int(r/1000)
    r -= M*1000
    C = int(r/100)
    r -= C*100
    D = int(r/10)
    r -= D*10
    U = r
    if U + D+ C +M == 10:
        L[ind,:] = [M, C , D , U]
        ind+=1
        if D == 9 and U ==1:
            L[ind,:] = [0, 0, 10, 0]
            ind+=1
        if C == 9 and D ==1:
            L[ind,:] = [0, 10, 0, 0]
            ind+=1
        if M == 9 and C ==1:
            L[ind,:] = [10, 0, 0, 0]
            ind+=1

L/=10  
L+= 0.0001
print(L)

In [ ]:
data = np.zeros((datanorm.shape[0], 286, 4))

for k in range(datanorm.shape[0]):
        datatemp = datanorm[k, :][:,np.newaxis]
       
        data[k,:,:] = datatemp * L
        


In [ ]:
limval = int(dataraw.shape[0]*.99)

Xtrain = data[:limval,:,:]
Ytrain = pd.get_dummies(dataraw["target"].iloc[:limval]).to_numpy()
Xval = data[limval:dataraw.shape[0],:,:]
Yval = pd.get_dummies(dataraw["target"].iloc[limval:]).to_numpy()


In [ ]:
import tensorflow as tf

Xtrainnew = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2], 1))

Xvalnew = np.reshape(Xval, (Xval.shape[0], Xval.shape[1], Xtrain.shape[2], 1))
N = 20
Ypredvect = np.zeros((datarawtest.shape[0],N))

kernel = 4
for i in range(N):
    print("model number " + str(i+1))
    tf.keras.backend.clear_session()
    initializer = tf.keras.initializers.GlorotNormal(seed = i)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (kernel, 4), activation = 'tanh',padding = 'same', input_shape = (286, 4, 1), kernel_initializer=initializer),
        tf.keras.layers.AveragePooling2D((kernel,1),kernel,padding = 'same'),
        tf.keras.layers.Conv2D(32, (kernel, 1), activation = 'tanh',padding = 'same', kernel_initializer=initializer),
        
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dropout(0.2),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Dense(512, activation = "relu", kernel_initializer=initializer),
        tf.keras.layers.Dropout(0.1),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512, activation = "relu", kernel_initializer=initializer),
        #tf.keras.layers.Dropout(0.1),
        #tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Dense(128, activation = "tanh", kernel_initializer=initializer),
        #tf.keras.layers.Dropout(0.1),
        #tf.keras.layers.BatchNormalization(),
        
        #tf.keras.layers.Dense(10, activation = "tanh", kernel_initializer=initializer),

        tf.keras.layers.Dense(10, activation = "sigmoid", kernel_initializer=initializer)
    ])
    #model.summary()
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer = optimizer, loss = loss, metrics=['accuracy'] )
    model.fit(Xtrainnew, Ytrain, epochs = 20, shuffle = True, validation_data = (Xvalnew, Yval), batch_size = 512)

    datatest = data[dataraw.shape[0]:,:,:]
    Xtest = np.reshape(datatest, (datatest.shape[0], datatest.shape[1], datatest.shape[2], 1))
    Ypred = model.predict(Xtest)
    Ypredvect[:,i] = Ypred.argmax(axis = 1)
    #answer = namelist[Ypredvect]


In [ ]:
L = [i for i in range(N)]
print(L)
for name in range(10):
    
        print(namelist[name])
        print("train")
        print(data[dataraw[dataraw.target==name].index,:,:].shape)
        print("test")
        L2 = np.copy(L)
        for j in L2:
            
            X = datatest[Ypredvect[:,j]==name,:,:].shape[0]    
            print( "modelel num " + str(j) + " : " + str(X) + " elements")
            if abs(X-10000) > 3000:
                del L[L.index(j)]
        print(L)         
    

In [ ]:
from scipy import stats

blender = stats.mode(Ypredvect[:,L], axis = 1)[0]
answers = namelist[blender.astype(int)]
for name in range(10):
    
        print(namelist[name])
        print(str(blender[blender==name].shape[0]) + " elements")

In [ ]:
print(answers)

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

submission['target'] = answers

In [ ]:
submission.to_csv("./submission.csv", index = False)